In [2]:
import pandas as pd
from lib.appearances_processor import AppearancesProcessor

processor = AppearancesProcessor(raw_file_path='../data/raw/appearances.json')

processor.process()

In [7]:
df = processor.prep_df
df[(df['player_id'] == 432671) & (df['date'] == '2021-02-13')]

,competition,round,date,player_position,goals,assists,yellow_cards,red_cards,minutes_played,home_club_id,away_club_id,player_id,player_name,player_club_id,game_id,appearance_id,home_club_goals,away_club_goals,season
6205,RLSW,23,2021-02-13,nan,0,0,0,0,90,983,1193,432671,luca-philipp,983,2826,6206,0,4.0,2020
6206,RLSW,23,2021-02-13,Goalkeeper,0,0,0,0,90,983,1193,432671,luca-philipp,983,2826,6206,0,4.0,2020


In [9]:
df = processor.raw_df
df['player_id'] == df['parent'].apply(lambda x: get_href_part(x['href'], 4)).astype('int32')

,type,href,parent,competition_code,matchday,date,venue,for,opponent,result,pos,goals,assists,yellow_cards,second_yellow_cards,red_cards,minutes_played
0,appearance,/steffen-tigges/leistungsdaten/spieler/296879,"{'type': 'player', 'href': '/steffen-tigges/pr...",RLW3,1,2020-09-05,A,"{'type': 'club', 'href': '/borussia-dortmund-i...","{'type': 'club', 'href': '/alemannia-aachen/sp...",0:1,CF,1,,,,,90'
1,appearance,/steffen-tigges/leistungsdaten/spieler/296879,"{'type': 'player', 'href': '/steffen-tigges/pr...",RLW3,2,2020-09-12,H,"{'type': 'club', 'href': '/borussia-dortmund-i...","{'type': 'club', 'href': '/1-fc-koln-ii/spielp...",2:1,CF,,,,,,90'
2,appearance,/steffen-tigges/leistungsdaten/spieler/296879,"{'type': 'player', 'href': '/steffen-tigges/pr...",RLW3,3,2020-09-16,A,"{'type': 'club', 'href': '/borussia-dortmund-i...","{'type': 'club', 'href': '/sportfreunde-lotte/...",0:2,CF,2,,,,,90'
3,appearance,/steffen-tigges/leistungsdaten/spieler/296879,"{'type': 'player', 'href': '/steffen-tigges/pr...",RLW3,4,2020-09-20,H,"{'type': 'club', 'href': '/borussia-dortmund-i...","{'type': 'club', 'href': '/rot-weiss-essen/spi...",1:1,CF,,,29',,,90'
4,appearance,/steffen-tigges/leistungsdaten/spieler/296879,"{'type': 'player', 'href': '/steffen-tigges/pr...",RLW3,5,2020-09-23,A,"{'type': 'club', 'href': '/borussia-dortmund-i...","{'type': 'club', 'href': '/sc-wiedenbruck-2000...",1:4,CF,1,,,,,90'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96650,appearance,/jamie-hamilton/leistungsdaten/spieler/567118,"{'type': 'player', 'href': '/jamie-hamilton/pr...",SC1,24,2021-01-16,H,"{'type': 'club', 'href': '/hamilton-academical...","{'type': 'club', 'href': '/dundee-united-fc/sp...",0:0,CB,,,87',,,90'
96651,appearance,/jamie-hamilton/leistungsdaten/spieler/567118,"{'type': 'player', 'href': '/jamie-hamilton/pr...",SC1,26,2021-01-27,A,"{'type': 'club', 'href': '/hamilton-academical...","{'type': 'club', 'href': '/celtic-glasgow/spie...",2:0,CB,,,,,,90'
96652,appearance,/jamie-hamilton/leistungsdaten/spieler/567118,"{'type': 'player', 'href': '/jamie-hamilton/pr...",SC1,27,2021-02-03,H,"{'type': 'club', 'href': '/hamilton-academical...","{'type': 'club', 'href': '/ross-county-fc/spie...",1:2,CB,,,,,,90'
96653,appearance,/jamie-hamilton/leistungsdaten/spieler/567118,"{'type': 'player', 'href': '/jamie-hamilton/pr...",SC1,28,2021-02-07,H,"{'type': 'club', 'href': '/hamilton-academical...","{'type': 'club', 'href': '/glasgow-rangers/spi...",1:1,CB,,,,,,90'
